# Liriscat paper experiments
### 1. Init
#### 1.1. Import libraries (necessary)

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["PYTHONHASHSEED"] = "0"
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":4096:8"

import liriscat
liriscat.utils.set_seed(0)

import logging
import gc
import json
import torch
liriscat.utils.set_seed(0)
import pandas as pd
from importlib import reload

#### 1.2. Set up the loggers (recommended)

In [2]:
liriscat.utils.setuplogger(verbose = True, log_name="liriscat", debug=False)

### 2. CDM prediction
#### 2.1. Training and testing, sequential version

In [3]:
import warnings

gc.collect()
torch.cuda.empty_cache()

In [ ]:
config = liriscat.utils.generate_eval_config(load_params=True, esc = 'error', valid_metric= 'mi_acc', pred_metrics = ["mi_acc"], profile_metrics = ['meta_doa'], save_params=False, n_query=6, num_epochs=100, batch_size=512)
liriscat.utils.set_seed(config["seed"])

config["dataset_name"] = "math2"
logging.info(config["dataset_name"])
config['learning_rate'] = 0.0001
config['inner_user_lr'] = 0.016848380924625605
config['lambda'] = 9.972254466547545e-06

config['meta_lr'] = 0.5
config['patience'] = 20#5
config['num_inner_users_epochs'] = 3
config['d_in'] = 4
config['num_responses'] = 12

#pred_metrics,df_interp = test(config)

CUDA is available. Using GPU.
[INFO 13:20] math2


In [5]:
logging.info(f'#### {config["dataset_name"]} ####')
logging.info(f'#### config : {config} ####')
config['embs_path']='../embs/'
config['params_path']='../ckpt/'

gc.collect()
torch.cuda.empty_cache()

# Dataset downloading for doa and rm
warnings.filterwarnings("ignore", message="invalid value encountered in divide")
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Concept map format : {question_id : [category_id1, category_id2, ...]}
concept_map = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_concept_map.json', 'r'))
concept_map = {int(k): [int(x) for x in v] for k, v in concept_map.items()}

## Metadata map format : {"num_user_id": ..., "num_item_id": ..., "num_dimension_id": ...}
metadata = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_metadata.json', 'r'))


## Tensor containing the nb of modalities per question
nb_modalities = torch.load(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_nb_modalities.pkl',weights_only=True)


[INFO 13:20] #### math2 ####
[INFO 13:20] #### config : {'seed': 0, 'dataset_name': 'math2', 'load_params': True, 'save_params': False, 'embs_path': '../embs/', 'params_path': '../ckpt/', 'early_stopping': True, 'esc': 'error', 'verbose_early_stopping': False, 'disable_tqdm': False, 'valid_metric': 'mi_acc', 'learning_rate': 0.0001, 'batch_size': 512, 'valid_batch_size': 10000, 'num_epochs': 100, 'eval_freq': 1, 'patience': 20, 'device': device(type='cuda'), 'lambda': 9.972254466547545e-06, 'tensorboard': False, 'flush_freq': True, 'pred_metrics': ['mi_acc'], 'profile_metrics': ['meta_doa'], 'num_responses': 12, 'low_mem': False, 'n_query': 6, 'CDM': 'impact', 'i_fold': 0, 'num_inner_users_epochs': 3, 'num_inner_epochs': 10, 'inner_lr': 0.0001, 'inner_user_lr': 0.016848380924625605, 'meta_trainer': 'Adam', 'num_workers': 0, 'pin_memory': False, 'debug': False, 'd_in': 4} ####


In [ ]:
meta_trainers = ['Approx_GAP']
for meta_trainer in meta_trainers : 
    config['meta_trainer'] = meta_trainer
    logging.info(f'#### meta_trainer : {config["meta_trainer"]} ####')
    for i_fold in range(1,2) : 
        config['i_fold'] = i_fold
            
        logging.info(f'#### i_fold : {i_fold} ####')
        ## Dataframe columns : (user_id, question_id, response, category_id)
        train_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_train_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})
        valid_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_valid_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})
        test_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_test_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})

        train_data = liriscat.dataset.CATDataset(train_df, concept_map, metadata, config,nb_modalities)
        valid_data = liriscat.dataset.EvalDataset(valid_df, concept_map, metadata, config,nb_modalities)
        test_data = liriscat.dataset.EvalDataset(test_df, concept_map, metadata, config,nb_modalities)

        for seed in range(1) :
            config['seed'] = seed
            logging.info(f'#### seed : {seed} ####')

            train_data.reset_rng()
            valid_data.reset_rng()
            test_data.reset_rng()

            S = liriscat.selectionStrategy.Random(metadata,**config)
            S.init_models(train_data, valid_data)
            S.train(train_data, valid_data)
            liriscat.utils.set_seed(0)
            S.reset_rng()
            d = (S.evaluate_test(test_data, train_data, valid_data))
            logging.info(d)
            logging.info(liriscat.utils.pareto_index(d))

    torch.cuda.empty_cache()

[INFO 13:20] #### meta_trainer : Approx_GAP ####
[INFO 13:20] #### i_fold : 1 ####
[INFO 13:29] #### seed : 0 ####
[INFO 13:29] Random_cont_model
[INFO 13:29] compiling CDM model
[INFO 13:31] compiling selection model
[INFO 13:31] ------- START Training
[INFO 13:31] train on cuda


/home/arthur/liriscat/liriscat/selectionStrategy/random.py:30: UserWarning: get_params() Notimplemented
  warnings.warn('get_params() Notimplemented')
  0%|          | 0/100 [00:00<?, ?it/s]

[INFO 13:32] ----- User batch : 0
[INFO 13:37] ----- User batch : 1
[INFO 13:40] ----- User batch : 2
[INFO 13:43] ----- User batch : 3
[INFO 13:46] ----- User batch : 4
[INFO 13:48] - meta_params gn: [1.0539125204086304]
[INFO 13:48] - meta_params norm: [34.032470703125]
[INFO 13:48] - meta_params: 0.5
[INFO 13:54] rmse : 0.6702445149421692
[INFO 13:54] valid_rmse : 0.857090175151825
[INFO 13:54] valid_loss : 1.2114529609680176


  1%|          | 1/100 [00:22<36:33, 22.16s/it]

[INFO 13:55] ----- User batch : 0
[INFO 13:58] ----- User batch : 1
[INFO 14:02] ----- User batch : 2
[INFO 14:05] ----- User batch : 3
[INFO 14:09] ----- User batch : 4
[INFO 14:10] - meta_params gn: [0.999675452709198]
[INFO 14:10] - meta_params norm: [35.73713684082031]
[INFO 14:10] - meta_params: 0.5
[INFO 14:15] rmse : 0.6698727011680603
[INFO 14:15] valid_rmse : 0.8566147089004517
[INFO 14:15] valid_loss : 1.2094014883041382


...

[INFO 29:49] ----- User batch : 0
[INFO 29:53] ----- User batch : 1
[INFO 29:56] ----- User batch : 2
[INFO 29:59] ----- User batch : 3
[INFO 30:03] ----- User batch : 4
[INFO 30:04] - meta_params gn: [0.5804523825645447]
[INFO 30:04] - meta_params norm: [92.8545913696289]
[INFO 30:04] - meta_params: 0.015625
[INFO 30:09] rmse : 0.6526936292648315
[INFO 30:09] valid_rmse : 0.8346465826034546
[INFO 30:09] valid_loss : 1.1744719743728638


 48%|████▊     | 48/100 [16:36<17:09, 19.79s/it]

[INFO 30:09] ----- User batch : 0
[INFO 30:13] ----- User batch : 1
[INFO 30:16] ----- User batch : 2
[INFO 30:20] ----- User batch : 3
[INFO 30:23] ----- User batch : 4
[INFO 30:25] - meta_params gn: [0.6483296155929565]
[INFO 30:25] - meta_params norm: [92.90681457519531]
[INFO 30:25] - meta_params: 0.015625
[INFO 30:29] rmse : 0.6521590352058411
[INFO 30:29] valid_rmse : 0.8339629173278809
[INFO 30:29] valid_loss : 1.1745935678482056


 49%|████▉     | 49/100 [16:57<16:58, 19.98s/it]

[INFO 30:29] ----- User batch : 0
[INFO 30:33] ----- User batch : 1
[INFO 30:36] ----- User batch : 2
[INFO 30:39] ----- User batch : 3


In [ ]:
config['meta_trainer'] = 'Approx_GAP'
#config['meta_trainer'] = 'Adam'
for seed in range(1) :
    config['seed'] = seed
    logging.info(f'#### seed : {seed} ####')
    
    train_data.reset_rng()
    valid_data.reset_rng()
    test_data.reset_rng()


    S = liriscat.selectionStrategy.Random(metadata,**config)
    S.init_models(train_data, valid_data)
    S.train(train_data, valid_data)
    liriscat.utils.set_seed(0)
    S.reset_rng()
    d = (S.evaluate_test(test_data, train_data, valid_data))
    print(liriscat.utils.pareto_index(d))
    print(d)
    torch.cuda.empty_cache()